### Add OpenCVSharp Package

In [0]:
#r "OpenCvSharp.dll"

In [0]:
#r "PlotNET.dll"

In [2]:
using System;
using OpenCvSharp;
using OpenCvSharp.Dnn;
using System.IO;
using System.Linq;
using System.Collections.Generic;
using PlotNET;

### Step1  Set YOLOv3 Class Label

In [3]:
var plotter = new Plotter();

In [4]:
plotter.ShowImage(@"dataset/opencvdataset/course3.png");

In [5]:
var classNames = File.ReadAllLines(@"dataset/opencvdataset/yolov3.txt").ToArray();

In [6]:
Scalar[] Colors = Enumerable.Repeat(false,classNames.Length).Select(x => Scalar.RandomColor()).ToArray();

### Step2 OpenCV to Load YOLOv3 Model

In [7]:
var net = CvDnn.ReadNet(@"dataset/opencvdataset/yolov3.weights",@"dataset/opencvdataset/yolov3.cfg");

### Step3 OpenCV Load TestImage

In [8]:
var img = Cv2.ImRead(@"dataset/opencvdataset/course3.png");

In [9]:
var w = img.Width;
var h = img.Height;

In [10]:
var blob = CvDnn.BlobFromImage(img,0.00392,new Size(416, 416), new Scalar(), true,false);

net.SetInput(blob,"data");

var layer_names =net.GetLayerNames();

var outlayer = net.GetUnconnectedOutLayers();

IList<string>  output_layers= new List<string>();

foreach(var item in outlayer)
{
                    //var temp = item-1;
    output_layers.Add(layer_names[item-1].ToString());
}

var outlayernames = net.GetUnconnectedOutLayersNames();

Mat[] outs =outlayernames.Select(_=>new Mat()).ToArray();

net.Forward(outs,outlayernames);

var threshold =0.3;
const int prefix = 5; 

foreach(var prob in outs)
{
    var rows = prob.Rows;
    for(int i =0 ; i<rows;i++)
    {
        var confidence =  prob.At<float>(i,4);
        if (confidence > threshold)
        {
            Cv2.MinMaxLoc(prob.Row[i].ColRange(prefix, prob.Cols), out _, out Point max);
            var classes = max.X;
            var probability = prob.At<float>(i, classes + prefix);
            if (probability > threshold) //more accuracy
            {
                    var centerX = prob.At<float>(i, 0) * w;
                    var centerY = prob.At<float>(i, 1) * h;
                    var width = prob.At<float>(i, 2) * w;
                    var height = prob.At<float>(i, 3) * h;
                    var label = $"{classNames[classes]} {probability * 100:0.00}%";
                    Console.WriteLine(label);
                    
                    var x1 = (centerX - width / 2) < 0 ? 0 : centerX - width / 2;
                    img.Rectangle(new Point(x1, centerY - height / 2), new Point(centerX + width / 2, centerY + height / 2), Colors[classes], 2);
                    var textSize = Cv2.GetTextSize(label, HersheyFonts.HersheyTriplex, 0.5, 1, out var baseline);
                    Cv2.Rectangle(img, new Rect(new Point(x1, centerY - height / 2 - textSize.Height - baseline),
                    new Size(textSize.Width, textSize.Height + baseline)), Colors[classes], Cv2.FILLED);
                    Cv2.PutText(img, label, new Point(x1, centerY - height / 2-baseline), HersheyFonts.HersheyTriplex, 0.5, Scalar.Black);
            }
        }
     }
}

person 65.10%

book 54.26%

person 73.06%

person 46.71%

person 51.81%

...

bottle 42.94%

keyboard 48.26%

cell phone 68.64%

mouse 45.00%

book 31.21%

In [11]:
img.SaveImage(@"dataset/opencvdataset/course3Result.jpg");

In [12]:
plotter.ShowImage(@"dataset/opencvdataset/course3Result.jpg");